In [1]:
import numpy as np
import random
import re
import sklearn
import time
from rdkit import Chem
import matplotlib.pyplot as plt
from sklearn.model_selection import GroupKFold
from sklearn.multioutput import MultiOutputClassifier
import pandas as pd
import os
import rdkit
from sklearn.ensemble import RandomForestClassifier
import os.path as osp
import sys
import pickle
import warnings
random.seed(4)
np.random.seed(4)
b22=True
dino=False
cellprof=False
def rowN_remover(x):
    return x.split("_")[0]
mesh_ohe_matrix=pd.read_csv("annotations/b22mesh_ohe_matrix.csv",index_col=0)
bmoa_ohe_matrix=pd.read_csv("annotations/b22_bmoa_ohematrix_ID2Name.csv",index_col=0,delimiter=";")
btarget_ohe_matrix=pd.read_csv("annotations/b22_btarget_ohematrix_ID2Name.csv",index_col=0,delimiter=";")
emb_file="embeddings/allemb_supbs40unsup40_bbbc22_5CJ_supcon_nolabANDnonans+_neighaug_lr_2-5_full_meshupd.csv"
if dino:
    emb_file='embeddings/emb_bs192_b22_5Ctiff_DINO+_nobrightcont.csv'
if cellprof:
    emb_file="embeddings/full_cellprofb22_nucnanfilt.csv"
if not dino or cellprof:
    con_emb=True
if con_emb:
    test_dfX0=pd.read_csv(emb_file,index_col=0,delimiter=";")
else:
    if cellprof:
        test_dfX0=pd.read_csv(emb_file,index_col=0,delimiter=";")
    if dino:
        test_dfX0=pd.read_csv(emb_file,header=None,index_col=0,delimiter=";")
#print(len(test_dfX0))
idkey="Molecules"
annos_list=["mesh","bmoa","btarget"]
if cellprof:
    test_dfX0.index=test_dfX0.Image_Metadata_SOURCE_COMPOUND_NAME
if b22:
    if con_emb:
        new_index=[rowN_remover(i) for i in test_dfX0.index]
        test_dfX0[idkey]=new_index#dft.index
    else:
        test_dfX0[idkey]=test_dfX0.index
    groups = [df for _, df in test_dfX0.groupby(idkey)]
    random.shuffle(groups)
    test_dfX0=pd.concat(groups).reset_index(drop=True)
test_dfX0.head(1)

,0,1,2,3,4,5,6,7,8,9,...,441,442,443,444,445,446,447,448,449,Molecules
0,0.015029,-0.110598,-0.026616,-0.022095,-0.050077,-0.057743,-0.072501,-0.028893,-0.010603,0.015269,...,-0.002973,0.082652,0.115795,-0.087896,0.09408,0.020165,0.036446,-1,0.001605,N-(4-BROMOBENZYL)-5-METHOXYTRYPTAMINE


In [2]:
for aoi in annos_list:#annotation of interest

    if aoi=="mesh":
        moi=mesh_ohe_matrix.copy()
    if aoi=="bmoa":
        moi=bmoa_ohe_matrix.copy()
    if aoi=="btarget":
        moi=btarget_ohe_matrix.copy()
    #select matrix of interest
    print(aoi)
    t0=time.time()

    skf = GroupKFold(n_splits=5)
    dX=test_dfX0.copy()
    if dino:
        start_feat,end_feat=2,-1
    if con_emb:
        start_feat,end_feat=0,224#225:447 is embedding for replicate, 449 is contrastive entropy
    if cellprof:
        start_feat,end_feat=10,10+824

    results=[]
    #print(emb_file)
    FullReporter={}
    test_dfX_2=test_dfX0.copy()
    test_dfX_2=test_dfX_2[test_dfX_2[idkey].isin(moi.index)]
    moi=moi[moi.index.isin(test_dfX_2[idkey])]
    print(len(test_dfX_2[idkey].unique()),len(test_dfX_2.index))
    dX=test_dfX_2.copy()
    for i, (train_index, test_index) in enumerate(skf.split(dX, groups=dX[idkey])):
        trainX=dX.iloc[train_index].copy()
        train_feat=trainX.iloc[:,start_feat:end_feat]
        train_labels=moi.loc[trainX.Molecules]
        testX=dX.iloc[test_index].copy()
        test_feat=testX.iloc[:,start_feat:end_feat]
        test_labels=moi.loc[testX.Molecules]
        forest = RandomForestClassifier(random_state=1,n_jobs=20)
        multi_target_forest = MultiOutputClassifier(forest, n_jobs=1)
        multi_target_forest.fit(train_feat, train_labels)
        score=multi_target_forest.score(test_feat,test_labels)
        pred=multi_target_forest.predict(test_feat)
        proba=multi_target_forest.predict_proba(test_feat)
        FullReporter["CV"+str(i+1)]=[pred,test_labels,score,proba]
        results.append(score)
        #print("Test Accuracy: ", score)
    print("Mean Accuracy over all cross validation folds: ",np.mean(results), "% +/- ", np.std(results))





mesh
838 30240
Mean Accuracy over all cross validation folds:  0.2113095238095238 % +/-  0.027208997924238372
bmoa
773 27936
Mean Accuracy over all cross validation folds:  0.0967328370554177 % +/-  0.011743056241709598
btarget
587 21240
Mean Accuracy over all cross validation folds:  0.03629943502824859 % +/-  0.019617237915203017
